In [0]:
%run "./Imports"

In [0]:
%run "./Model_Tuning"

In [0]:
%run "./General_Functions"

### Nerdearla 2021 - E2E Unified Demand Planning: Demand Forecasting

This notebook contains the code of the Demand Forecasting Pipeline, which is the process used to tune (train +
validation) and back-test the models for each product (SKU) according to the different defined experiments. In this
context, an experiment is a scenario composed of:

  * Algorithm, such as Prophet, SARIMAX, XGBoost.
  * Set of input features (excluded from the scope of the workshop)

For each product this pipeline will train, tune, yield a best model (best set of hyperparameters) and backtest it for
each of the different experiments; which means that every product will have as many "best models" as the number of
experiments. The decision about which of these models to use as the final model is done by selecting the better on in
terms of the validation WAPE.

The result of this process consists of logging for all products the best model of each experiment into the Mlflow
tracking API and generating the forecast for the back-testing period with each one of these, after that only the
forecast corresponding to the best model among all the experiments is kept.

The functions included are:

| Function | Description |
| -------- | ----------- |
| `obtain_models` | for each product, obtains the best forecasting model per experiment |

###### Initializing variables

In [0]:
# Experiment variables
algorithms = ["prophet", "sarimax"]
holidays = False
num_evals = 5

# Dates for validation
start_val = "2019-01-01"
end_val = "2019-06-30"

# Dates for testing
start_test = "2019-07-01"
end_test = "2019-12-31"

###### Defining search space of each algorithm

In [0]:
# Defining search space for prophet
params_prophet = {
    "changepoint_prior_scale":  hp.loguniform("changepoint_prior_scale", np.log(0.001), np.log(0.5)),
    "seasonality_prior_scale": hp.loguniform("seasonality_prior_scale", np.log(0.01), np.log(10)),
    "holidays_prior_scale": hp.loguniform("holidays_prior_scale", np.log(0.01), np.log(10)),
}

# Defining search space for sarimax
params_sarimax = {
    "p": hp.choice("p", [0, 1, 2]),
    "d": hp.choice("d", [0, 1]),
    "q": hp.choice("q", [0, 1, 2]),
    "P": hp.choice("P", [0, 1, 2]),
    "D": hp.choice("D", [0, 1]),
    "Q": hp.choice("Q", [0, 1, 2]),
    "s": 12
}

###### Setting Mlflow experiment

In [0]:
# Defining experiment path
mlflow_exp = r"/UDP_E2E_Forecasting/nerdearla_udp_consumption"

# Launching Mlflow client
client = MlflowClient()

# Creating experiment or re-using it if already exists
experiment = client.get_experiment_by_name(mlflow_exp)
if experiment is None:
    exp_id = mlflow.create_experiment(mlflow_exp)
else:
    exp_id = experiment.experiment_id

###### Defining modeling function

In [0]:
def obtain_models(data, df_frds):
    """
    For each product, obtains the best forecasting model per experiment; where an experiment is defined by an algorithm
    in the context of the workshop.

    Obtaining the best model of each experiment is done by performing hyperparameter tuning, which involves training and
    validating multiple sets of hyperparameters to then select the best performing set according to a specific metric on
    the validation set. Finally, all the best models from the experiments are used to generate the forecast for the
    back-testing set and their performances on that set are recorded.

    Results for each experiment such as train WAPE, validation WAPE and test WAPE are logged into Mlflow.

    Parameters
    __________
        data (pd.DataFrame): Dataset with the time series of the product.
        df_frds (pd.DataFrame): Dataset with holidays.

    Returns
    _______
        df_forecasts (pd.DataFrame): Table with the forecasts of the best models for the back-testing set.
    """
    # Ensuring order of observations
    data = data.sort_values(by="ds", ascending=True).reset_index(drop=True)

    # Obtaining product info
    sku = data["n_sku"][0]

    # Splitting the series
    df_trainval, df_test = split_series(data, start_test, end_test)

    # Defining the output object
    df_forecasts = pd.DataFrame()

    # Looping over the algorithms
    for algorithm in algorithms:
        # Validating the algorithm to use
        if algorithm == "sarimax":
            search_space = params_sarimax
        elif algorithm == "prophet":
            search_space = params_prophet

        # Tuning the model
        results = tune_ts_model(
            algorithm, search_space, num_evals, df_trainval, start_val, end_val, holidays=holidays, df_frds=None
        )

        # Re-fitting model and generating forecast
        df_fcst_alg, test_wape = refit_generate_forecast(
            algorithm, results["params"], df_trainval, df_test, holidays, df_frds
        )

        # Appending forecast to the output object
        df_forecasts = df_forecasts.append(df_fcst_alg)

        # Starting run and assigning tags
        mlflow.start_run(experiment_id=exp_id, run_name=str(sku))
        mlflow.set_tags(
           {
            "experiment": "Nerdearla 2021",
            "product": sku,
            "algorithm": algorithm
           }
        )

        # Logging results in Mlflow
        mlflow.log_metrics({"train_wape": results["train_wape"], "val_wape": results["val_wape"], "test_wape": test_wape})

        # Ending run
        mlflow.end_run()

    # Adding identification column
    df_forecasts["n_sku"] = sku

    return df_forecasts

##### Demand forecasting pipeline main code

###### 1. Loading the data from DBFS

In [0]:
# Loading demand data from DBFS 
df_data = spark.read.csv(r"/FileStore/tables/test_file/raw_consumption_data_clean.csv", sep=',', header=True, inferSchema=True)
df_data = df_data.withColumn("ds", to_date(df_data["ds"], "yyyy-MM-dd"))

feriados = pd.DataFrame({"ds": ["1949-03-01", "1954-03-01", "1959-03-01", "1964-03-01"], "holiday": ["a", "a", "a", "a"]})
feriados["ds"] = pd.to_datetime(feriados["ds"])

###### 2. Performing modeling of SKUs

In [0]:
# Defining schema of the resulting dataframe:
result_schema = StructType(
    [
     StructField("algorithm", StringType(), False),
     StructField("ds", DateType(), False),
     StructField("fcst", FloatType(), False),
     StructField("n_sku", IntegerType(), False)
    ]
)

# Performing modeling of the DFUs
df_fcsts = df_data.groupBy("n_sku") \
    .applyInPandas(
        lambda df: obtain_models(df, feriados),
        result_schema
    ) \
    .persist(StorageLevel.MEMORY_ONLY)

# Adding identification key of experiments
df_fcsts = df_fcsts.withColumn("exp_key", concat(df_fcsts["n_sku"], lit("_"), df_fcsts["algorithm"]))
rows = df_fcsts.count()

###### 3. Selecting best experiment per SKU

In [0]:
# Loading experiment results as a Spark DataFrame
df_exp = spark.read.format("mlflow-experiment").load(exp_id)
df_exp = df_exp.select("tags.product", "tags.algorithm", "metrics.train_wape", "metrics.val_wape", "metrics.test_wape").toPandas()

# Creating identification key of experiments
df_exp["exp_key"] = df_exp["product"] + "_" + df_exp["algorithm"]

# Obtaining best result per SKU
best_exps = df_exp.groupby(by=["product"]).agg({"val_wape": "min"}).reset_index()
best_exps["best"] = 1

# Filtering the best result
df_exp = pd.merge(df_exp, best_exps, on=["product", "val_wape"], how="left")
df_exp = df_exp[df_exp["best"] == 1]
list_best = list(df_exp["exp_key"])

###### 4. Filtering forecasts of the best experiments and saving on Delta

In [0]:
# Filtering forecasts of the best experiments
df_fcsts = df_fcsts.filter(df_fcsts["exp_key"].isin(list_best))

# Writing results to Delta
df_fcsts.write.mode("overwrite")\
    .format("delta") \
    .option("overwriteSchema", "true") \
    .save("dbfs:/FileStore/results/demand_forecasts")